In [1]:
include("../src/param_sweeps.jl")

run_all_interventions_variableprev_scenario_pairs (generic function with 2 methods)

In [4]:
Occ = repeat(ParcelOccPattern,10)
NP = Int64.(round.(repeat(NPparcel*ParcelPkgPattern,10)));

In [25]:
Ninf = 0
i = 0
out = Dict()
while Ninf < 8
    out = run_sim_delivery_wp(BasicParcelParams, Occ, NP; PkgParams=BasicPkgParams)
    susc = sum(out["Susceptible"],dims=1)
    Ninf = susc[1] - susc[end]
    print(i,' ',Ninf,'\n')
    i += 1
end

0 1
1 1
2 1
3 1
4 1
5 2
6 1
7 1
8 0
9 1
10 22


In [143]:

df = DataFrame([Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,
                Int64,Int64,Int64,Int64,Int64,Int64,Int64,
                Int64,Int64],
               [:Day,:S_D,:S_L,:S_O,:S_T,:I_D,:I_L,:I_O,:I_T,
                :R_D,:R_L,:R_O,:R_T,:Q_D,:Q_L,:Q_O,:Q_T], length(time))
df["Day"] = out["time"]
df["S_D"] = out["Susceptible"][1,:]
df["S_L"] = out["Susceptible"][2,:]
df["S_O"] = out["Susceptible"][3,:]
df["S_T"] = df["S_D"]+df["S_L"]+df["S_O"]
df["I_D"] = out["Infectious"][1,:]
df["I_L"] = out["Infectious"][2,:]
df["I_O"] = out["Infectious"][3,:]
df["I_T"] = df["I_D"]+df["I_L"]+df["I_O"]
df["R_D"] = out["Recovered"][1,:]
df["R_L"] = out["Recovered"][2,:]
df["R_O"] = out["Recovered"][3,:]
df["R_T"] = df["R_D"]+df["R_L"]+df["R_O"]
df["Q_D"] = out["Isolated"][1,:]
df["Q_L"] = out["Isolated"][2,:]
df["Q_O"] = out["Isolated"][3,:]
df["Q_T"] = df["Q_D"]+df["Q_L"]+df["Q_O"]
CSV.write("example_outbreak.csv",df)

"example_outbreak.csv"

Visualise the network

In [31]:
Incidence = zeros(length(Occ))
Prevalence = zeros(length(Occ))
Params = BasicParcelParams
PairParams = DefaultPairParams
TestParams = DefaultTestParams
sim = initialise(Params, PairParams, Incidence, Prevalence )
sim_summary, i_day, Go = setup_delivery_wp_model!(sim, Params, TestParams, Occ)

(Dict{Any,Any}("Isolated" => [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0],"CustomersInfected" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0],"InfsByType" => [[0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0]],"IndexCasePeakVL" => 9.324717095636691,"IsolatedDueToContactFalsePos" => [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0],"NewContactTruePosIsolators" => [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0],"NewTestSympIsolators" => [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0],"NewIsolators" => [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0],"NewContactFalsePosIsolators" => [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0],"IsolatedDueToFalsePos" => [0 0 … 0 0; 0 0 … 0 0; 0 0 … 0 0]…), 7, true)

In [124]:
layout=(args...)->spring_layout(args...; C=7.0)
labels=['D','P','O']
colors=[colorant"blue",colorant"red",colorant"green"]
ew = zeros(ne(sim["cohort_network"]))
for (i,e) in enumerate(edges(sim["cohort_network"]))
    ew[i] = get_prop(sim["cohort_network"],src(e),dst(e),:weight)
end
cr = range(RGB(colorant"black"), stop=RGB(colorant"yellow"), length=100)
ec = cr[Int8.(round.(ew*10000))]

using Cairo, Compose
draw(PDF("cohort_graph.pdf", 8cm, 8cm), gplot(sim["cohort_network"], NODESIZE=0.035, 
        nodestrokec=colorant"black", NODELABELSIZE=2.0, nodestrokelw=1.0, 
        nodefillc=colors[sim["job"]], nodelabel=labels[sim["job"]], 
        nodelabelc=colorant"white", layout=layout))

In [119]:
range(RGB(colorant"black"), stop=RGB(colorant"yellow"), length=100)

In [112]:
?gplot

search: gplot gplothtml pgfplots pgfplotsx GraphPlot generate_peak_viral_load



Given a graph and two vectors of X and Y coordinates, returns a Compose tree of the graph layout

**Arguments**

`G` Graph to draw

`layout` Optional. Layout algorithm. Currently can be one of [`random_layout`, `circular_layout`, `spring_layout`, `shell_layout`, `stressmajorize_layout`, `spectral_layout`]. Default: `spring_layout`

`locs_x, locs_y` Locations of the nodes. Can be any units you want, but will be normalized and centered anyway

`NODESIZE` Optional. Max size for the nodes. Default: `3.0/sqrt(N)`

`nodesize` Optional. Relative size for the nodes, can be a Vector. Default: `1.0`

`nodelabel` Optional. Labels for the vertices, a Vector or nothing. Default: `nothing`

`nodelabelc` Optional. Color for the node labels, can be a Vector. Default: `colorant"black"`

`nodelabeldist` Optional. Distances for the node labels from center of nodes. Default: `0.0`

`nodelabelangleoffset` Optional. Angle offset for the node labels. Default: `π/4.0`

`NODELABELSIZE` Optional. Largest fontsize for the vertice labels. Default: `4.0`

`nodelabelsize` Optional. Relative fontsize for the vertice labels, can be a Vector. Default: `1.0`

`nodefillc` Optional. Color to fill the nodes with, can be a Vector. Default: `colorant"turquoise"`

`nodestrokec` Optional. Color for the nodes stroke, can be a Vector. Default: `nothing`

`nodestrokelw` Optional. Line width for the nodes stroke, can be a Vector. Default: `0.0`

`edgelabel` Optional. Labels for the edges, a Vector or nothing. Default: `[]`

`edgelabelc` Optional. Color for the edge labels, can be a Vector. Default: `colorant"black"`

`edgelabeldistx, edgelabeldisty` Optional. Distance for the edge label from center of edge. Default: `0.0`

`EDGELABELSIZE` Optional. Largest fontsize for the edge labels. Default: `4.0`

`edgelabelsize` Optional. Relative fontsize for the edge labels, can be a Vector. Default: `1.0`

`EDGELINEWIDTH` Optional. Max line width for the edges. Default: `0.25/sqrt(N)`

`edgelinewidth` Optional. Relative line width for the edges, can be a Vector. Default: `1.0`

`edgestrokec` Optional. Color for the edge strokes, can be a Vector. Default: `colorant"lightgray"`

`arrowlengthfrac` Optional. Fraction of line length to use for arrows. Equal to 0 for undirected graphs. Default: `0.1` for the directed graphs

`arrowangleoffset` Optional. Angular width in radians for the arrows. Default: `π/9 (20 degrees)`
